# Estimation d'intervalles de prédiction pour la prévision de séries temporelles

Dans cet exemple, nous utilisons MAPIE pour estimer les intervalles de prédiction associés aux prévision de séries temporelles. Nous utilisons ici l'approche standard de validation croisée pour estimer les résidus et les intervalles de prédiction associés.

Nous utilisons ici le dataset de consommation d'électricité de Victoria utilisé dans le livre "Forecasting : Principles and Practice" par R. J. Hyndman et G. Athanasopoulos.
La demande d'électricité présente des variations saisonnières quotidiennes et hebdomadaires et est influencée par la températur, considérée ici comme une variable exogène.

Les données sont modélisées par un modèle de type Random Forest optimisé par un `RandomizedSearchCV` en utilisant une validation croisée séquentielle `TimeSeriesSplit`. Le meilleur modèle est ensuite introduit dans `MapieRegressor` pour estimer les intervalles de prédiction associés. Nous considérons deux stratégies, avec les méthodes de rééchantillonnage CV et CV+ et en utilisant une méthode standard `KFold` pour estimer les résidus. La garantie mathématique de la validation croisée séquentielle `TimeSeriesSplit` dans MAPIE pour l'estimation des intervalles de prédiction n'est pas encore validée et sera donc implémentée ultérieurement.

In [ ]:
import pandas as pd
from scipy.stats import randint
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_absolute_percentage_error
from mapie.estimators import MapieRegressor
from mapie.metrics import coverage_score

Commençons par charger les données et extraire les variables temporelles.

In [ ]:
demand_df = pd.read_csv(
    "../data/demand_temperature.csv",
    parse_dates=True,
    index_col=0
)
demand_df["Date"] = pd.to_datetime(demand_df.index)
demand_df["Weekofyear"] = demand_df.Date.dt.isocalendar().week.astype('int64')
demand_df["Weekday"] = demand_df.Date.dt.isocalendar().day.astype('int64')
demand_df["Hour"] = demand_df.index.hour

Séparons le jeu de données en jeux d'entraînement et de test, le jeu de test étant sur les deux dernières semaines.

In [ ]:
num_forecast_steps = 24 * 7 * 2
demand_train = demand_df.iloc[:-num_forecast_steps, :].copy()
demand_test = demand_df.iloc[-num_forecast_steps:, :].copy()
X_train = demand_train.loc[:, ["Weekofyear", "Weekday", "Hour", "Temperature"]]
y_train = demand_train["Demand"]
X_test = demand_test.loc[:, ["Weekofyear", "Weekday", "Hour", "Temperature"]]
y_test = demand_test["Demand"]

Définissons les paramètres d'entrée.

In [ ]:
n_iter = 10
n_splits = 5
random_state = 59
rf_model = RandomForestRegressor(random_state=random_state)
rf_params = {
    "max_depth": randint(2, 30),
    "n_estimators": randint(10, 1e3)
}
alpha = 0.1

**Exercice.** Optimisons le modèle Random Forest par une étude de paramètres `RandomizedSearchCV` mais cette fois ci en considérant une validation croisée séquentielle [`TimeSeriesSplit`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html) afin que chaque jeu d'entraînement soit toujours antérieure au jeu de validation.

In [ ]:
tscv = TimeSeriesSplit(n_splits=n_splits)
cv_obj = RandomizedSearchCV(
    rf_model,  # correction
    param_distributions=rf_params,  # correction
    n_iter=n_iter,
    cv=tscv,  # correction
    scoring="neg_root_mean_squared_error",
    random_state=random_state,
    verbose=0,
    n_jobs=-1,
)
cv_obj.fit(X_train, y_train)
best_est = cv_obj.best_estimator_

**Exercice.** Estimons ensuite les intervalles de prédiction associées au meilleur modèle Random Forest avec les stratégies CV et CV+. Calculons ensuite les couvertures effectives et les largeurs moyennes des intervalles.

In [ ]:
strategies = {
    "cv": dict(method="base", cv=n_splits),
    "cv_plus": dict(method="plus", cv=n_splits),
}
y_pred, y_pis, coverages, widths = {}, {}, {}, {}
for strategy, params in strategies.items():
    mapie = MapieRegressor(best_est, **params)
    mapie.fit(X_train, y_train)
    y_pred_, y_pis_ = mapie.predict(X_test, alpha=alpha)  # correction
    y_pred[strategy] = y_pred_
    y_pis[strategy] = y_pis_
    coverages[strategy] = coverage_score(
        y_test, y_pis_[:, 0, 0], y_pis_[:, 1, 0]  # correction
    )
    widths[strategy] = (
        y_pis_[:, 1, 0] - y_pis_[:, 0, 0]  # correction
    ).mean()

Comparons maintenant les résultats avec les deux stratégies.

In [ ]:
for strategy in strategies:
    print(
        "Coverage and prediction interval width mean for the "
        f"{strategy:8} strategy: "
        f"{coverages[strategy]:.3f}, {widths[strategy]:.3f}"
    )

Visualisons enfin les intervalles de prédiction sur nos données de test.

In [ ]:
fig = go.Figure()
# lower/upper bounds
fig.add_trace(go.Scatter(
    name="lower bound",
    x=demand_test.index,
    y=y_pis["cv_plus"][:, 0, 0],
    mode="lines",
    line=dict(color="#1f77b4", dash='solid', width=0.1)
))
fig.add_trace(go.Scatter(
    name="upper bound",
    x=demand_test.index,
    y=y_pis["cv_plus"][:, 1, 0],
    mode="lines",
    fill="tonexty",
    line=dict(color="#1f77b4", dash='solid', width=0.1)
))
# predictions
fig.add_trace(go.Scatter(
    name="predictions",
    x=demand_test.index,
    y=y_pred["cv_plus"],
    mode="lines",
    line=dict(color="#1f77b4", dash='solid')
))
# data
fig.add_trace(go.Scatter(
    name="lower bound",
    x=demand_test.index,
    y=demand_test.Demand,
    mode="lines",
    line=dict(color="#ff7f0e", dash='solid')
))
fig.update_layout(
    autosize=False,
    width=800,
    height=400,
    yaxis_title="Hourly demand (GW)",
    hovermode="x",
    showlegend=False
)
fig.show()